# Python Map/Reduce nad MongoDB (nekretnine.listings)
Ovaj notebook demonstrira 3 smislenе map/reduce obrade u Pythonu (streaming preko MongoDB cursora), kao zamjenu za deprecated MongoDB mapReduce.

## 1) Setup

In [2]:
import pymongo
from pymongo import MongoClient
from collections import defaultdict
import statistics
import pandas as pd
from pathlib import Path
from IPython.display import display

OUT_DIR = Path("../podaci/obradeni")
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [3]:
client = MongoClient("mongodb://localhost:27017")
db = client["nekretnine"]
collection = db["listings"]

collection.estimated_document_count()

107208

## 2) Helper: validacija i rekonstrukcija `perM2`
- U podacima postoje outlier vrijednosti (npr. `2147483647`) i zapisi s `livingM2 = 0` koji predstavljaju greške nastale pri scrape-u stranice ili jednostavno namjerno loše psotavljenje od strane oglasitelja.
- Preferira se izračun `amount / livingM2` ako je moguć; inače se koristi spremljeni `price.perM2` ako je u granicama.

In [4]:
def get_valid_per_m2(doc, lo=100, hi=20000):
    """Vrati validan perM2 (float) ili None."""
    amount = doc.get("price", {}).get("amount")
    living = doc.get("area", {}).get("livingM2")
    stored = doc.get("price", {}).get("perM2")

    try:
        if amount is not None and living is not None and float(living) > 0:
            per = float(amount) / float(living)
            if lo <= per <= hi:
                return per
    except Exception:
        pass

    try:
        per = float(stored)
        if per != 2147483647 and lo <= per <= hi: #maksimalna vrijednost 32-bitnog potpisanog cijelog broja 2147483647
            return per
    except Exception:
        pass

    return None

## 3) Map/Reduce 1 — Prosječna cijena po m² po gradu
**Map:** emitira `(city, perM2)` za validne zapise.
**Reduce:** računa aritmetičku sredinu po gradu.

In [5]:
def map_city_per_m2(doc):
    city = doc.get("location", {}).get("city")
    if not city:
        return None
    per = get_valid_per_m2(doc)
    if per is None:
        return None
    return (city, per)

def reduce_average(values):
    return sum(values) / len(values)

In [6]:
mapped = defaultdict(list)
kept = dropped = 0

cursor = collection.find({}, {
    "location.city": 1,
    "price.amount": 1,
    "price.perM2": 1,
    "area.livingM2": 1
})

for doc in cursor:
    out = map_city_per_m2(doc)
    if out is None:
        dropped += 1
        continue
    city, per = out
    mapped[city].append(per)
    kept += 1

avg_per_m2_by_city = {city: reduce_average(vals) for city, vals in mapped.items()}
print("kept:", kept, "| dropped:", dropped)

kept: 103212 | dropped: 3996


In [7]:
df_avg_city = (
    pd.DataFrame(list(avg_per_m2_by_city.items()), columns=["city", "avg_per_m2"])
      .sort_values("avg_per_m2", ascending=False)
      .reset_index(drop=True)
)
display(df_avg_city.head(10))


city   avg_per_m2
0            Dubrovnik  6605.921618
1              Opatija  6078.863983
2     Donji Kukuruzari  5932.764654
3            Lokvičići  5900.000000
4                Split  5775.613998
5  Dubrovnik - Okolica  5192.238672
6               Rovinj  5058.811366
7  Dubrovačko Primorje  5057.484394
8                 Hvar  5055.563287
9                  Bol  4924.399226

In [8]:
print("Broj gradova u rezultatu:", len(df_avg_city))

Broj gradova u rezultatu: 556


## 4) Agreggation pipeline — Broj oglasa po (grad, tip nekretnine)


In [9]:
pipeline_count_city_type = [
    {"$match": {"location.city": {"$ne": None}, "propertyType": {"$ne": None}}},
    {
        "$group": {
            "_id": {"city": "$location.city", "propertyType": "$propertyType"},
            "count": {"$sum": 1},
        }
    },
    {"$sort": {"count": -1}},
]

res_count_city_type = list(collection.aggregate(pipeline_count_city_type, allowDiskUse=True))

df_count_city_type = (
    pd.DataFrame(res_count_city_type)
      .assign(
          city=lambda d: d["_id"].apply(lambda x: x.get("city")),
          propertyType=lambda d: d["_id"].apply(lambda x: x.get("propertyType")),
      )
      .drop(columns=["_id"])
      .sort_values(["count", "city", "propertyType"], ascending=[False, True, True])
      .reset_index(drop=True)
)

display(df_count_city_type.head(10))

count        city propertyType
0   5947      Rijeka    apartment
1   4153        Pula    apartment
2   3063       Poreč        house
3   2861       Poreč    apartment
4   2656     Opatija    apartment
5   2232  Crikvenica    apartment
6   1860     Medulin    apartment
7   1685       Zadar    apartment
8   1682        Umag    apartment
9   1621       Labin        house

## 5) Aggregation pipeline — Medijan cijene po m² po tipu nekretnine



In [10]:
PER_LO = 100
PER_HI = 20000
SENTINEL = 2147483647

per_calc_expr = {
    "$cond": [
        {
            "$and": [
                {"$ne": ["$price.amount", None]},
                {"$ne": ["$area.livingM2", None]},
                {"$gt": ["$area.livingM2", 0]},
            ]
        },
        {"$divide": ["$price.amount", "$area.livingM2"]},
        None,
    ]
}

per_final_expr = {"$ifNull": ["$perCalc", "$price.perM2"]}

pipeline_median_type = [
    {"$addFields": {"perCalc": per_calc_expr}},
    {"$addFields": {"perM2Final": per_final_expr}},
    {
        "$match": {
            "propertyType": {"$ne": None},
            "perM2Final": {"$nin": [None, SENTINEL], "$gte": PER_LO, "$lte": PER_HI}
        }
    },
    {
        "$group": {
            "_id": "$propertyType",
            "median_per_m2": {"$percentile": {"input": "$perM2Final", "p": [0.5], "method": "approximate"}},
            "n": {"$sum": 1},
        }
    },
    {"$addFields": {"median_per_m2": {"$arrayElemAt": ["$median_per_m2", 0]}}},
    {"$sort": {"median_per_m2": -1}},
]

try:
    res_median_type = list(collection.aggregate(pipeline_median_type, allowDiskUse=True))
    df_median_type = (
        pd.DataFrame(res_median_type)
          .rename(columns={"_id": "propertyType"})
          .sort_values("median_per_m2", ascending=False)
          .reset_index(drop=True)
    )
    display(df_median_type)
except Exception as e:
    print("MongoDB ne podržava $percentile ili je onemogućen. Fallback na Python median.")
    print("Detalj:", type(e).__name__, str(e)[:200])

    cursor = collection.find(
        {},
        {"propertyType": 1, "price.amount": 1, "price.perM2": 1, "area.livingM2": 1},
    )

    mapped = defaultdict(list)
    kept = dropped = 0

    for doc in cursor:
        ptype = doc.get("propertyType")
        if not ptype:
            dropped += 1
            continue
        per = get_valid_per_m2(doc, lo=PER_LO, hi=PER_HI)
        if per is None:
            dropped += 1
            continue
        mapped[ptype].append(per)
        kept += 1

    df_median_type = (
        pd.DataFrame([(k, statistics.median(v), len(v)) for k, v in mapped.items()],
                     columns=["propertyType", "median_per_m2", "n"])
          .sort_values("median_per_m2", ascending=False)
          .reset_index(drop=True)
    )
    print("kept:", kept, "| dropped:", dropped)
    display(df_median_type)

propertyType  median_per_m2      n
0    apartment    3661.963879  56267
1        house    2800.126597  47123

In [11]:
df_avg_city.to_csv(OUT_DIR / "avg_per_m2_by_city.csv", index=False, encoding="utf-8")
df_count_city_type.to_csv(OUT_DIR / "count_by_city_propertyType.csv", index=False, encoding="utf-8")
df_median_type.to_csv(OUT_DIR / "median_per_m2_by_propertyType.csv", index=False, encoding="utf-8")

OUT_DIR.resolve()

WindowsPath('C:/Users/jurej/Desktop/Semestar7/Teorija baza podataka/jure_jelcic/Projekt/podaci/obradeni')

## 6) MapReduce 2: prosječni  broj pregleda po gradu


In [21]:

OUT = "mr_avg_views_by_city"

map_fn = """
function () {
  var city = this.location && this.location.city ? this.location.city : null;
  var views = this.stats && this.stats.views != null ? Number(this.stats.views) : null;

  if (!city) return;
  if (views === null || !isFinite(views) || views < 0) return;

  emit(city, {sum: views, n: 1});
}
"""

reduce_fn = """
function (key, values) {
  var out = {sum: 0, n: 0};
  values.forEach(function (v) {
    out.sum += v.sum;
    out.n += v.n;
  });
  return out;
}
"""

finalize_fn = """
function (key, reduced) {
  reduced.avg_views = reduced.n ? (reduced.sum / reduced.n) : null;
  return reduced;
}
"""

query = {"location.city": {"$ne": None}, "stats.views": {"$ne": None}}

db = collection.database
db.command({
    "mapReduce": collection.name,
    "map": map_fn,
    "reduce": reduce_fn,
    "finalize": finalize_fn,
    "query": query,
    "out": OUT
})

res = list(db[OUT].find({}, {"_id": 1, "value.avg_views": 1, "value.n": 1}))

df = pd.DataFrame(res)
df["city"] = df["_id"]
df["avg_views"] = df["value"].apply(lambda x: x.get("avg_views"))
df["n"] = df["value"].apply(lambda x: x.get("n"))
df = df[["city", "avg_views", "n"]].sort_values(["avg_views", "n"], ascending=[False, False]).reset_index(drop=True)

display(df.head(15))


city    avg_views     n
0          Rovišće  9410.800000   5.0
1            Berek  8876.500000   4.0
2    Breznički Hum  7388.000000   1.0
3        Šandrovac  6957.250000   4.0
4      Ernestinovo  6720.000000   2.0
5   Grubišno Polje  6240.142857   7.0
6           Garčin  5163.750000   4.0
7            Lovas  4975.000000   1.0
8       Pleternica  4661.666667   6.0
9          Dubrava  4511.133333  15.0
10         Slatina  4363.600000   5.0
11      Martijanec  4361.000000   2.0
12        Žumberak  3824.500000   6.0
13   Ferdinandovac  3821.500000   2.0
14      Lipovljani  3674.333333   3.0

## 7)  Brze provjere kvalitete podataka
Ove ćelije služe za identifikaciju ekstremnih `perM2` vrijednosti i tipičnih izvora problema (npr. `livingM2=0`).

In [ ]:
cursor = collection.find(
    {"price.perM2": {"$exists": True, "$ne": None}},
    {"location.city": 1, "price.perM2": 1, "price.amount": 1, "area": 1}
).sort("price.perM2", -1).limit(10)

for d in cursor:
    print(d.get("location", {}).get("city"),
          "perM2=", d.get("price", {}).get("perM2"),
          "amount=", d.get("price", {}).get("amount"),
          "area=", d.get("area"))

Našice perM2= 2147483647 amount= 150000 area= {'livingM2': 0, 'gardenM2': None}
Požega perM2= 2147483647 amount= 200000 area= {'livingM2': 0, 'gardenM2': 571}
Marina perM2= 2147483647 amount= 830000 area= {'livingM2': 0, 'gardenM2': 605}
Labin perM2= 3150000 amount= 3150000 area= {'livingM2': 1, 'gardenM2': 1000}
Rogoznica perM2= 2600000 amount= 2600000 area= {'livingM2': 1, 'gardenM2': None}
Oroslavje perM2= 1500000 amount= 1500000 area= {'livingM2': 1, 'gardenM2': None}
Kostrena perM2= 1400000 amount= 1400000 area= {'livingM2': 1, 'gardenM2': 2100}
Opatija perM2= 1300000 amount= 1300000 area= {'livingM2': 1, 'gardenM2': None}
Trogir perM2= 1200000 amount= 1200000 area= {'livingM2': 1, 'gardenM2': None}
Rijeka perM2= 850000 amount= 850000 area= {'livingM2': 1, 'gardenM2': 6000}


## 8) Primjeri za isprobavanje

**Napomena** , ako želite isprobati primjere to jest gledati statistike za druge gradove morate pokrenuti sve skripte koje su u folderu , potrebno je neko vrijeme da se podaci učitaju , 

In [ ]:
CITY = "Zadar" # samo zamijenite grad npr. SPlit

display(
    df_avg_city[df_avg_city["city"].str.lower() == CITY.lower()]
)

city   avg_per_m2     n
66  Zadar  3796.853064  1969

In [ ]:
display( # top 5 gradova po cijeni m2
    df_avg_city.sort_values("avg_per_m2", ascending=False).head(5)
)

city   avg_per_m2     n
0         Dubrovnik  6605.921618   128
1           Opatija  6078.863983  2761
2  Donji Kukuruzari  5932.764654     3
3         Lokvičići  5900.000000     1
4             Split  5775.613998   902

In [ ]:
CITY = "Split" #Broj oglasa po tipu nekretnine za odabrani grad

display(
    df_count_city_type[df_count_city_type["city"].str.lower() == CITY.lower()]
      .sort_values("count", ascending=False)
)


count   city propertyType
20     900  Split    apartment
166    123  Split        house

In [ ]:
display( # grad s najviše oglasa
    df_count_city_type
        .groupby("city", as_index=False)["count"]
        .sum()
        .sort_values("count", ascending=False)
        .head(5)
)


city  count
370      Rijeka   6657
339       Poreč   5924
355        Pula   5108
53   Crikvenica   3721
248     Medulin   3318

In [ ]:
display(df_median_type)

propertyType  median_per_m2      n
0    apartment    3661.963879  56267
1        house    2800.126597  47123

In [ ]:
PTYPES = ["apartment", "house"] # filtriranje po tipu

display(
    df_median_type[df_median_type["propertyType"].isin(PTYPES)]
)


propertyType  median_per_m2      n
0    apartment    3661.963879  56267
1        house    2800.126597  47123